In [1]:
import matplotlib
matplotlib.use('Agg')

import numpy as np
#Pan Sharpening Demo
import skimage
import skimage.io
import skimage.transform

from skimage import data
from skimage.viewer import ImageViewer

%matplotlib inline

In [2]:
#Channels, 2 blue, 3 green, 4 red
def sharpen(root):
    path="/Users/yamaoka/landsat/downloads/"+root+"/clipped/"

    r=skimage.io.imread(path+root+"_B4.TIF", as_grey=True) #Load Red
    g=skimage.io.imread(path+root+"_B3.TIF", as_grey=True) #Load Green
    b=skimage.io.imread(path+root+"_B2.TIF", as_grey=True) #Load Blue
    ir=skimage.io.imread(path+root+"_B5.TIF", as_grey=True) #Band 5 - Near Infrared
    
    #ndvi process 
    ndvi = np.true_divide((ir - r), (ir + r))
    rgb= np.dstack((r,g,b)) #Combine into correctly ordered stack

    landsat_highres=path+root+"_B8.TIF"
    pan=skimage.io.imread(landsat_highres, as_grey=True)/65535 #Load chromatic, normalize to between 0 and 1


    rgb_big=skimage.transform.resize(rgb, output_shape=(pan.shape[0],pan.shape[1],3), order=3, mode='constant', cval=0.0) #Resize the rgb composite to match the chromatic
    ndvi_big=skimage.transform.resize(ndvi, output_shape=(pan.shape[0],pan.shape[1]), order=3, mode='constant', cval=0.0) #Resize the rgb composite to match the chromatic

    hsv = skimage.color.rgb2hsv(rgb_big) #Conver the rgb to the HSV colorspace
    hsv[...,2]=pan #Poor man's pan sharpening, replace the intensity channel with the observed chromatic image
    rgb_pan = skimage.color.hsv2rgb(hsv) #Convert back to rgb space
    return rgb_pan, ndvi_big 


In [3]:
landsat1, ndvi=sharpen(root="LC80270392017030LGN00") #File name of your imagery

In [4]:
#viewer = ImageViewer(ndvi)
#viewer.show()

In [5]:
from scipy import ndimage as ndi
from math import floor
landsat1_dog = landsat1 - ndi.gaussian_filter(landsat1, 15) #Difference of gaussians

In [6]:
stack=np.dstack((landsat1, landsat1_dog, ndvi))
#stack=np.dstack((landsat1, landsat1_dog))

depth=stack.shape[2]
edge=3
buff=floor(edge/2)
stack_window=skimage.util.view_as_windows(np.pad(stack,pad_width=((buff,buff), (buff,buff),(0,0)), mode ='minimum'), window_shape=(edge,edge,depth), step=1) 
print('stack_window.shape ')
print(stack_window.shape) #3 by 3 moving window around each pixel

stack_flat=stack_window.reshape(-1,edge*edge*depth) 
print('stack_flat.shape')
print(stack_flat.shape) #Flatten to rows

stack_window.shape 
(3117, 2451, 1, 3, 3, 7)
stack_flat.shape
(7639767, 63)


In [7]:
#make labels
building= skimage.io.imread('/Users/yamaoka/Dropbox/Insight/landuse/labels/austin_tx_buildings_labels_allt.TIF', as_grey=True) #Load Buildings
landuse = skimage.io.imread('/Users/yamaoka/Dropbox/Insight/landuse/labels/austin_tx_landuse_labels_noallt.TIF', as_grey=True) #Load Landuse tmp

In [8]:
#viewer = ImageViewer(building)
#viewer.show()

In [9]:
ll = landuse.flatten() 
lb = building.flatten() 

print(ll[ll>1.0])
print(lb[0])

[55537 55537 55537 ..., 55537 55537 55537]
55537


In [10]:
nodata = 55537

ll[ll != nodata] = 1.
lb[lb != nodata] = 2.

ll[ll == nodata] = 0.
lb[lb == nodata] = 0.

labels = ll+lb
labels[labels==3] = 2. #i know this is stupid need to fix later

print(labels[labels!=0].shape)

(2424080,)


In [11]:
#test_reshape=labels.reshape(landuse.shape)
#print(test_reshape.shape)
#labels_predicted_andknown[labelsfixed>0]=labelsfixed[labelsfixed>0]
#finalcolors=np.array([[0,0,0], #0 unlabeled
#                     [32,66,239], #1 water
#                     [0,102,0], #2 vegitation
#                     [102,51,0], #3 buildings
#                     [0,0,0], #4 rail
#                     [128,128,128]],dtype=np.int64)#5 road
#img = finalcolors[test_reshape]
#outpath = "out/path/here"
#skimage.io.imsave("rf_predictions.png", labels_predicted)
#skimage.io.imsave("/Users/yamaoka/Desktop/test.png", img)

In [12]:
x_tmp=stack_flat
y_tmp=labels

x_tmp = stack_flat[labels>0,...]
y_tmp = labels[labels>0]

y_tmp = y_tmp.astype(int)

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_tmp, y_tmp, test_size=0.99, random_state=42)

proportions= np.bincount(y_train)
smallest=min(proportions[1:])
weights = smallest/proportions
weights = weights[y_train]

/Users/yamaoka/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  


In [14]:
print(proportions)
print(x_train.shape)
print(y_train)
print(weights[y_train==1])

[    0  9538 14702]
(24240, 63)
[1 1 1 ..., 2 1 2]
[ 1.  1.  1. ...,  1.  1.  1.]


In [15]:
from sklearn.svm import SVC
rf = SVC(gamma=0.001, C=100)
rf.fit(x_train,y_train, sample_weight=weights)

rf.fit(x_train,y_train, sample_weight=weights)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [16]:
rf.score(x_test,y_test)

0.68198463230882056

In [17]:
x_test_big, x_test_small, y_test_big, y_test_small = train_test_split(x_test, y_test, test_size=0.01, random_state=42)

In [18]:
x_check_bld = x_test_big[y_test_big==1,...]
y_check_bld = y_test_big[y_test_big==1]
print(x_check_bld.shape)

predictions=rf.predict(x_check_bld)

(936155, 63)


In [19]:
diff = predictions-y_check_bld

print(diff[diff==0].shape)
rf.score(x_check_bld,y_check_bld)

(630183,)


0.67316096159289862

In [20]:

from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, x_test_small, y_test_small, cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[ 0.68208333  0.705625    0.658125    0.70458333  0.67909981]
Accuracy: 0.69 (+/- 0.04)


In [21]:
#rf.oob_score_
from sklearn import metrics
f1scores = cross_val_score(rf, x_test_small, y_test_small, cv=5, scoring='f1_macro')
print(f1scores)
print("F1: %0.2f (+/- %0.2f)" % (f1scores.mean(), f1scores.std() * 2))

[ 0.59616729  0.64664839  0.53239201  0.64803386  0.57956616]
F1: 0.60 (+/- 0.09)


In [30]:
import pickle
filename='rf200_242ktrain_wnvdi.sav'
pickle.dump(rf, open(filename, 'wb'))

import pickle


In [ ]:
from sklearn.svm import SVC
clf = svm.SVC(gamma=0.001, C=100)
clf.fit(x_train,y_train, sample_weight=weights)
filename='svc001100_first.sav'
pickle.dump(clf, open(filename, 'wb'))

In [98]:
labelsfixed = labels.reshape(landuse.shape)
predictions=rf.predict(x_test) 

[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  5.9min
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed: 15.6min finished


NameError: name 'labels_predicted_andknown' is not defined

/Users/yamaoka/anaconda/envs/py35/lib/python3.5/site-packages/skimage/io/_io.py:132: UserWarning: rf_predictions.png is a low contrast image
  warn('%s is a low contrast image' % fname)


In [109]:
#labels_predicted=predictions.reshape(labelsfixed.shape)
#labels_predicted_andknown[labelsfixed>0]=labelsfixed[labelsfixed>0]
#inalcolors=np.array([[0,0,0], #0 unlabeled
#                     [32,66,239], #1 water
#                     [0,102,0], #2 vegitation
#                     [102,51,0], #3 buildings
#                     [0,0,0], #4 rail
#                     [128,128,128]])#5 road
#outpath = "out/path/here"
#skimage.io.imsave("rf_predictions.png", labels_predicted)
#skimage.io.imsave("rf_predictions_andknown.png", finalcolors[labels_predicted_andknown])testcolors=np.array([0,0.8,0.2]) 
#skimage.io.imsave("rf_predictions.png", testcolors[labels_predicted],)
#skimage.io.imsave("dog.png", landsat1_dog)

/Users/yamaoka/anaconda/envs/py35/lib/python3.5/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint16
  "%s to %s" % (dtypeobj_in, dtypeobj))
